In [2]:
import os
from pydantic import Extra
import requests
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
from langchain.prompts import PromptTemplate
# Run chain
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS # type: ignore
from langchain_ollama import OllamaEmbeddings # type: ignore
import warnings
warnings.filterwarnings('ignore')

In [3]:
os.environ["token"] = "Bearer token|123675a6-95f6-4fb7-bc95-30095472ae3a|02de311fd83421a7fd637bf34dc8f959caa29f39888d2919e4b9640a2220224b"
token = os.environ["token"]
Path = r"D:\OneDrive - Wipro\Desktop\Static Dumps\Role-Skill-Cluster Master Report.csv"

In [4]:
from typing import ClassVar

parser = StrOutputParser()

class LlamaLLM(LLM):
    llm_url: ClassVar[str] = 'https://api.lab45.ai/v1.1/skills/completion/query'
    
    backend:        Optional[str]   = 'gpt-35-turbo-16k'
    temp:           Optional[float] = 0.7
    top_p:          Optional[float] = 0.1
    top_k:          Optional[int]   = 40
    n_batch:        Optional[int]   = 8
    n_threads:      Optional[int]   = 4
    n_predict:      Optional[int]   = 256
    max_tokens:     Optional[int]   = 256
    repeat_last_n:  Optional[int]   = 64
    repeat_penalty: Optional[float] = 1.18

    class Config:
        extra = Extra.forbid

    @property
    def _llm_type(self) -> str:
        return "gpt-35-turbo-16k"
    
    @property
    def _get_model_default_parameters(self):
        return {
            "max_tokens": self.max_tokens,
            #"n_predict": self.n_predict,
            "top_k": self.top_k,
            "top_p": self.top_p,
            "temperature": self.temp,
            #"n_batch": self.n_batch,
            #"repeat_penalty": self.repeat_penalty,
            #"repeat_last_n": self.repeat_last_n,
        }

    def _call(
        self,
        prompt: str,
        user: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        payload = {
        "messages": [
            {
            "content": prompt,
            "role": user
            }
        ],
        "skill_parameters": {
            "model_name": "gpt-35-turbo-16k",
            "max_output_tokens": 4096,
            "temperature": 0,
            "top_k": 5
        },
        "stream_response": False
        }

        headers = {"Content-Type": "application/json","Authorization": token}

        response = requests.post(self.llm_url, json=payload, headers=headers, verify=False)

       # print("API Response:", response.json())
        response.raise_for_status()

        return response.json()  # get the response from the API

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "llmUrl": self.llm_url,
            'model_parameters': self._get_model_default_parameters
            }

In [5]:
llm = LlamaLLM()

In [6]:
ROLE_CLUSTER_MASTER = pd.read_csv(Path)
ROLE_CLUSTER_MASTER_REQD=ROLE_CLUSTER_MASTER[['CLUSTER_NAME','SKILL_NAME']].head(1000)

In [7]:
ROLE_CLUSTER_MASTER_REQD

,CLUSTER_NAME,SKILL_NAME
0,SENIOR CONSULTANT - TECH AND IMPL CONSUMER GOO...,"SSIS (L2), Application Testing (L2), Business ..."
1,LEAD ADMINISTRATOR VMWARE AWS CLOUD - L1,"VMware Server Admin (L3), Windows and Vmware I..."
2,Tools Engineer ASSET MANAGEMENT -SNOW - L3,"ITIL Asset-Config Mgmt (L1), Software Asset Ma..."
3,DOMAIN CONSULTANT CORE BANKING - L2,"Core Banking (L2), Payments (L1)"
4,TEST LEAD TEST DATA MANAGEMENT - L1,"Mainframe Testing (L1), Python Scripting (L1),..."
...,...,...
995,TECHNO FUNCTIONAL CONSULTANT BLUEYONDER - SUP...,"Consulting (L2), BY Supply Chain- Demand manag..."
996,LEAD ADMINISTRATOR CITRIX VDI - L1,"Citrix Admin (L3), Microsoft Active Directory ..."
997,TECHNICAL LEAD CORE JAVA - L1,"PL-SQL (L3), PL-SQL (L3), Core Java (L3), Core..."
998,TEST ENGINEER PRODUCT TEST AUTOMATION CLI - L2,"Python Scripting (L2), Test Scripting Language..."


In [ ]:
JD = []
for row in ROLE_CLUSTER_MASTER_REQD.itertuples():
 #print(row)
 skills=row.SKILL_NAME
 title=row.CLUSTER_NAME
 company="Wipro"
 user = "system"
 prompt = """ As a skilled Technical Interview Panel with expertise in technology and content writing , your role is to meticulously evaluate skills and title provided and give detailed job description having below points outlined.
            Use the mentioned skills along with proficiency  & title to generate the job description, where L1 means Beginner, L2 means Intermediate, L3 means Advanced, and L4 means Expert,
            Job Role: 
            Give skill specified by user
            Clearly state the role/title of the position and give brief summary of the role/title in 3-4 lines
            
            Key Responsibilities:
            Title/Role will have overall impact on responsibilities
            Utilize expertise in [Key Skill 1] and [title/Role] to [Responsibility 1] .
            Apply advanced knowledge of [Key Skill 2]  to [Responsibility 2].
            Demonstrate proficiency in [Key Skill 3] to [Responsibility 3].
            Utilize [Key Skill 4]  to [Responsibility 4].
            Apply [Key Skill 5] to [Responsibility 5].

            Qualifications and Skills:

            Proficiency in [Key Skill 1] is essential for this role, with a strong focus on [Specific Requirement].
            Advanced knowledge of [Key Skill 2] is highly desirable, with the ability to [Specific Requirement].
            Familiarity with [Key Skill 3] is beneficial, with the capacity to [Specific Requirement].
            Experience in [Key Skill 4] is advantageous, with the ability to [Specific Requirement].
            Proficiency in [Key Skill 5] is preferred, with the capability to [Specific Requirement].

            Experience Requirements:

            A minimum of [Number of Years] years of relevant experience in [Industry/Field].
            Previous experience in [Specific Type of Experience] is beneficial, with a focus on [Relevant Proficiency].
            Demonstrated proficiency in [Key Skill 1] and [Key Skill 2] through [Type of Experience].
            Proven track record of applying [Key Skill 3] and [Key Skill 4] in [Type of Experience].

            """ + skills + title + company # type: ignore
 result = llm._call(prompt,user)
 parsed_result = result['data']['content'] # type: ignore
 #print()
 JD.append("CLUSTER NAME :"+title +'\n'+"CLUSTER SKILLS : "+skills+'\n'+parsed_result+'\n'+'\n'+'\n'+'\n') # type: ignore

In [ ]:
#for item in JD:
#    print(item)

In [11]:
with open("outfilenew", "w") as outfile:
    outfile.write("\n".join(JD))